**Contexto y Necesidad del proyecto**

En nuestro entorno laboral, se generan PDFs diariamente con información relevante que incluye enlaces a páginas web, imágenes y redes sociales. Tradicionalmente, se ha utilizado un script monolítico para realizar todas las tareas de extracción de texto de las paginas web y de imagenes, lo que dificulta el mantenimiento, la ampliación y la escalabilidad del sistema. Este proyecto surge para:

- **Automatizar el flujo de trabajo:** Permitir que cualquier usuario suba un PDF y obtenga de forma automática un .json con todo el texto extraido.
- **Optimizar el procesamiento:** Utilizar técnicas de caché para evitar reprocesamientos, lo que reduce la carga en el sistema y mejora la eficiencia.
- **Preparar la integración con sistemas avanzados:** Los resultados (almacenados en formatos CSV y JSON) servirán como insumo para procesos de generación de embeddings y para la creación de un agente especializado RAG, permitiendo responder a consultas complejas basadas en la información extraída.
- **Facilitar futuras escalabilidades:** Una arquitectura modular, con cada componente definido a través de interfaces claras, permite que, si es necesario, se migre a una arquitectura de microservicios de manera sencilla sin reescribir la lógica de negocio.

**Tareas Principales del Proyecto**

1. **Extracción de URLs desde el PDF:**  
   - **Objetivo:** Abrir el PDF diario y extraer todos los enlaces junto con el contexto (ubicación en el documento y fragmentos de texto cercanos).  
   - **Salida:** Guardar los enlaces extraídos en un archivo CSV para facilitar su seguimiento.

2. **Clasificación de URLs:**  
   - **Objetivo:** Validar y clasificar los enlaces extraídos en distintas categorías:  
     - Páginas HTML  
     - Enlaces a imágenes  
     - Enlaces a redes sociales u otros  
   - **Salida:** Separar las URLs según su tipo para procesarlas de forma especializada.

3. **Scraping de contenido HTML:**  
   - **Objetivo:** Realizar solicitudes HTTP (o usar Selenium cuando sea necesario) a las URLs de páginas HTML.  
   - **Tareas Específicas:**  
     - Utilizar siempre el header `'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/91.0.4472.124'` para garantizar la consistencia con la ejecución diaria.  
     - Limpiar y extraer el contenido textual y metadatos (título, descripción).  
     - Calcular la relevancia del contenido en base a palabras clave configuradas.  
     - Implementar un sistema de caché, basado en una clave generada (por ejemplo, MD5 de la URL), que verifique de manera uniforme la expiración (24 horas o 86400 segundos) para evitar reprocesamientos innecesarios.

4. **Extracción de texto de imágenes:**  
   - **Objetivo:** Descargar las imágenes referenciadas en las URLs clasificadas como imágenes y extraer el texto que contienen mediante una API externa.  
   - **Tareas Específicas:**  
     - Aplicar un sistema de caché similar al del scraping HTML, utilizando una clave generada a partir de la imagen, para evitar llamadas redundantes a la API si el resultado ya está cacheado y es reciente.

5. **Consolidación y seguimiento de resultados:**  
   - **Objetivo:** Unificar los resultados obtenidos del scraping HTML y la extracción de texto de imágenes en un formato JSON consistente, que posteriormente servirá como insumo para la generación de embeddings y la creación de un agente RAG.  
   - **Tareas Específicas:**  
     - Actualizar un historial de URLs procesadas en un módulo independiente para evitar duplicidades en ejecuciones futuras, lo que favorece la reutilización y la escalabilidad.  
     - Generar estadísticas de procesamiento (por ejemplo, total de URLs, porcentaje de contenido relevante) para monitorear y evaluar el rendimiento del sistema.

Proyecto cuyo objetivo es desarrollar un aplicativo web que permita a los usuarios de mi centro de trabajo subir un PDF (generado diariamente) y extraer de él enlaces, que se utilizarán para las siguientes tareas:

1. **Extracción de URLs desde el PDF:**  
   - Abrir el PDF y extraer todos los enlaces, junto con el contexto y la ubicación en el documento.  
   - Guardar los enlaces extraídos en un archivo CSV para su seguimiento.

2. **Clasificación de URLs:**  
   - Validar y clasificar los enlaces extraídos en diferentes categorías (por ejemplo, páginas HTML, imágenes, redes sociales u otros).  
   - Separar las URLs de imágenes de las URLs de páginas HTML para procesarlas de manera independiente.

3. **Scraping de contenido HTML:**  
   - Realizar solicitudes HTTP (o usar Selenium cuando sea necesario) a las URLs de páginas HTML, utilizando siempre el header `'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/91.0.4472.124'` para asegurar la coherencia con la ejecución diaria.  
   - Limpiar y extraer el contenido textual y metadatos relevantes (como título y descripción).  
   - Calcular la relevancia del contenido basado en palabras clave.  
   - Implementar un sistema de caché basado en una clave generada (por ejemplo, mediante un hash MD5 de la URL) y verificar la expiración de forma uniforme (24 horas, es decir, 86400 segundos) para evitar reprocesamientos.

4. **Extracción de texto de imágenes:**  
   - Descargar las imágenes referenciadas en las URLs clasificadas como imágenes.  
   - Enviar las imágenes descargadas a una API externa para extraer el texto que contienen.  
   - Aplicar una estrategia de caché similar a la del scraping HTML (usando una clave generada a partir de la imagen) y verificar la expiración de forma uniforme para evitar llamadas innecesarias a la API.

5. **Consolidación y seguimiento de resultados:**  
   - Unificar los resultados extraídos del scraping HTML y la extracción de texto de imágenes en un formato consistente (JSON) que servirá como insumo para generar embeddings y construir un agente especializado basado en técnicas RAG.  
   - Actualizar el historial de URLs procesadas en un módulo independiente (por ejemplo, `history_tracker.py`) para evitar duplicidades en futuras ejecuciones, lo cual favorece la reutilización y escalabilidad.  
   - Generar estadísticas de procesamiento (por ejemplo, cantidad total de URLs, porcentaje de contenido relevante, etc.) para monitoreo.

Actualmente, dispongo de un script monolítico (py.txt) que implementa estas funciones, pero necesito modularizarlo de acuerdo a la siguiente estructura de proyecto:

```
scrap_1403/
│
├── codigo/                        
│   ├── main.py                    # Orquestador principal: integra todo el flujo (basado en py)
│   ├── lib/                       # Biblioteca de funciones modulares
│   │   ├── config_manager.py      # Gestión de configuración (load_config, get_paths)
│   │   ├── pdf_processor.py       # Extracción de enlaces y contexto desde PDFs; guardado en CSV
│   │   ├── url_manager.py         # Validación y clasificación de URLs (páginas HTML, imágenes, social, etc.)
│   │   ├── html_scraper.py        # Scraping de contenido HTML con caché y cálculo de relevancia (usando el User-Agent especificado)
│   │   ├── image_processor.py     # Descarga de imágenes y extracción de texto de imágenes mediante API externa, con caché
│   │   ├── api_client.py          # Cliente para la API externa de extracción de texto de imágenes
│   │   ├── cache_utils.py         # Funciones para generación, guardado y carga de caché (clave MD5, expiración uniforme)
│   │   ├── file_manager.py        # Gestión de archivos: lectura/escritura de CSV y JSON, creación de directorios
│   │   └── history_tracker.py     # Seguimiento y actualización del historial de URLs procesadas (módulo independiente)
│   │
│   └── tests/                     # Pruebas unitarias para cada módulo
│       ├── test_config_manager.py   
│       ├── test_pdf_processor.py  
│       ├── test_url_manager.py    
│       ├── test_html_scraper.py   
│       ├── test_image_processor.py
│       ├── test_api_client.py     
│       ├── test_cache_utils.py    
│       ├── test_file_manager.py   
│       └── test_history_tracker.py
│
├── credentials/                   # Credenciales y secretos
│   ├── api_keys.yaml              # Claves de API (ejemplo, para la API de extracción de texto de imágenes)
│   ├── credentials.env            # Archivo de variables de entorno (opcional)
│   └── .gitignore                 # Configurado para excluir credenciales y archivos sensibles
│
├── base/                          # Carpeta de PDFs diarios para procesar
│   └── [fecha].pdf                # Ejemplo: 01042025.pdf, 27022025.pdf, etc.
│
├── input/                         # Carpeta principal de datos procesados
│   ├── In/                        # Enlaces extraídos de los PDFs
│   │   └── links_extracted_[fecha].csv  # CSV con los enlaces extraídos de cada PDF
│   │
│   ├── Out/                       # Resultados del scraping
│   │   ├── scraped_texts_[fecha].json   # Textos extraídos de páginas HTML
│   │   └── Img_[fecha].json             # Textos extraídos de imágenes mediante API
│   │
│   ├── Images/                    # Datos de imágenes
│   │   ├── image_links_[fecha].json     # Metadatos de enlaces de imágenes extraídos del PDF
│   │   └── downloads/             # Imágenes descargadas organizadas por fecha
│   │       └── [fecha]/           # Carpeta para cada fecha
│   │           ├── img_1_[fecha].jpg    # Ejemplo: imagen descargada 1 del día
│   │           ├── img_2_[fecha].jpg    # Ejemplo: imagen descargada 2 del día
│   │           └── texto_imagenes_api.json  # Resultados del texto extraído de imágenes por API
│   │
│   ├── Social/                    # Datos relacionados con redes sociales (si se extraen)
│   │   └── social_links_[fecha].json    # Enlaces a redes sociales extraídos del PDF
│   │
│   └── Stats/                     # Estadísticas del procesamiento diario
│       └── stats_[fecha].json     # Estadísticas y métricas del día
│
└── requirements.txt               # Lista de dependencias para el proyecto (ej., PyMuPDF, Flask/FastAPI, requests, Selenium, etc.)
```



PC:
Se trabaja en una PC windows 11, en lenguaje python y se escribe el codigo en VS Code
Prompt

"Eres un desarrollador experto en Python y arquitecturas de software escalables. Tengo un proyecto cuyo objetivo es desarrollar un aplicativo web que permita a los usuarios de mi centro de trabajo subir un PDF (generado diariamente) y extraer de él enlaces que se usarán para las siguientes tareas:

1. **Extracción de URLs desde el PDF:**  
   - Abrir el PDF y extraer todos los enlaces, junto con el contexto y la ubicación en el documento.  
   - Guardar los enlaces extraídos en un archivo CSV para su seguimiento.

2. **Clasificación de URLs:**  
   - Validar y clasificar los enlaces extraídos en diferentes categorías (por ejemplo, HTML, imágenes, redes sociales u otros).  
   - Separar las URLs de imágenes y de páginas HTML para procesarlas por separado.

3. **Scraping de contenido HTML:**  
   - Realizar solicitudes HTTP (o usar Selenium en casos necesarios) a las URLs de páginas HTML, utilizando siempre el header `'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/91.0.4472.124'` para mantener la coherencia con la ejecución diaria.  
   - Limpiar y extraer el contenido textual y metadatos relevantes (como título y descripción).  
   - Calcular la relevancia del contenido basado en palabras clave.  
   - Implementar un sistema de caché eficiente basado en una clave generada (por ejemplo, mediante un hash MD5 de la URL), y verificar de forma uniforme la expiración del caché (definida en 86400 segundos, es decir, 24 horas) para evitar reprocesamientos dentro de ese período.

4. **Extracción de texto de imágenes:**  
   - Descargar las imágenes referenciadas en las URLs clasificadas como imágenes.  
   - Enviar las imágenes descargadas a una API externa para extraer el texto que contienen.  
   - Aplicar una estrategia de caché similar a la del scraping HTML (utilizando una clave generada a partir de la imagen) y verificar la expiración uniformemente para evitar llamadas innecesarias a la API.

5. **Consolidación y seguimiento de resultados:**  
   - Unificar los resultados extraídos del scraping HTML y de la extracción de texto de imágenes en un formato JSON consistente, optimizado para su posterior uso en la generación de embeddings y construcción de un agente especializado basado en técnicas RAG.  
   - Actualizar el historial de URLs procesadas en un módulo independiente (por ejemplo, `history_tracker.py`), lo que favorecerá la reutilización, las pruebas unitarias y la escalabilidad futura.  
   - Generar estadísticas de procesamiento (por ejemplo, cantidad total de URLs, porcentaje de contenido relevante, etc.) para su monitoreo.

El objetivo principal de esta refactorización es crear una arquitectura modular con componentes desacoplados mediante interfaces claras, lo que permitirá evolucionar fácilmente hacia una arquitectura de microservicios si las necesidades de escalabilidad lo requieren. El sistema debe permitir tanto la ejecución automática programada como la iniciada por usuarios a través de una interfaz web simple, donde cualquier usuario pueda subir un PDF y obtener automáticamente el JSON con todo el texto extraído.

Actualmente dispongo de un script monolítico (main4_g2.py) que implementa todas estas funciones, pero necesito modularizarlo de acuerdo a la siguiente estructura de proyecto:

```
scrap_1402/
│
├── codigo/
│   ├── main.py                    # Orquestador principal: integra todo el flujo (basado en main4_g2.py)
│   ├── lib/
│   │   ├── config_manager.py      # Gestión de configuración (load_config, get_paths)
│   │   ├── pdf_processor.py       # Extracción de enlaces y contexto desde PDFs; guardado en CSV
│   │   ├── url_manager.py         # Validación y clasificación de URLs (HTML, imágenes, social, etc.)
│   │   ├── html_scraper.py        # Scraping de contenido HTML con caché y cálculo de relevancia (usando el User-Agent especificado)
│   │   ├── image_processor.py     # Descarga de imágenes y extracción de texto mediante API externa, con caché
│   │   ├── api_client.py          # Cliente para la API externa de extracción de texto de imágenes
│   │   ├── cache_utils.py         # Funciones para generación, guardado y carga de caché (clave MD5, expiración uniforme)
│   │   ├── file_manager.py        # Gestión de archivos: lectura/escritura de CSV y JSON, creación de directorios
│   │   └── history_tracker.py     # Seguimiento y actualización del historial de URLs procesadas (módulo independiente)
│   │
│   └── tests/                     # Pruebas unitarias para cada módulo
│       ├── test_config_manager.py   
│       ├── test_pdf_processor.py  
│       ├── test_url_manager.py    
│       ├── test_html_scraper.py   
│       ├── test_image_processor.py
│       ├── test_api_client.py     
│       ├── test_cache_utils.py    
│       ├── test_file_manager.py   
│       └── test_history_tracker.py
│
├── credentials/                   # Credenciales y secretos
│   ├── api_keys.yaml              # Claves de API (ejemplo, para la API de extracción de texto de imágenes)
│   ├── credentials.env            # Archivo de variables de entorno (opcional)
│   └── .gitignore                 # Configurado para excluir credenciales y archivos sensibles
│
├── base/                          # Carpeta de PDFs diarios para procesar
│   └── [fecha].pdf                # Ejemplo: 01042025.pdf, 27022025.pdf, etc.
│
├── input/                         # Carpeta principal de datos procesados
│   ├── In/                        # Enlaces extraídos de los PDFs
│   │   └── links_extracted_[fecha].csv  # CSV con los enlaces extraídos de cada PDF
│   │
│   ├── Out/                       # Resultados del scraping
│   │   ├── scraped_texts_[fecha].json   # Textos extraídos de páginas HTML
│   │   └── Img_[fecha].json             # Textos extraídos de imágenes mediante API
│   │
│   ├── Images/                    # Datos de imágenes
│   │   ├── image_links_[fecha].json     # Metadatos de enlaces de imágenes extraídos del PDF
│   │   └── downloads/             # Imágenes descargadas organizadas por fecha
│   │       └── [fecha]/           # Carpeta para cada fecha
│   │           ├── img_1_[fecha].jpg    # Ejemplo: imagen descargada 1 del día
│   │           ├── img_2_[fecha].jpg    # Ejemplo: imagen descargada 2 del día
│   │           └── texto_imagenes_api.json  # Resultados de la extracción de texto de imágenes por API
│   │
│   ├── Social/                    # Datos relacionados con redes sociales (si se extraen)
│   │   └── social_links_[fecha].json    # Enlaces a redes sociales extraídos del PDF
│   │
│   └── Stats/                     # Estadísticas del procesamiento diario
│       └── stats_[fecha].json     # Estadísticas y métricas del día
│
└── requirements.txt               # Lista de dependencias para el proyecto (ej., PyMuPDF, Flask/FastAPI, requests, Selenium, etc.)
```

Ten en cuenta que:

- La configuración debe incluir un parámetro `cache_expiry` de 86400 (24 horas en segundos) para determinar el tiempo de validez del caché.
- En el módulo de scraping HTML se debe usar el header `'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/91.0.4472.124'` para reemplazar cualquier valor genérico o por defecto, manteniendo así la coherencia con la ejecución diaria.
- Tanto los módulos de scraping HTML como el de extracción de texto de imágenes deben implementar un sistema de caché uniforme (con funciones en `cache_utils.py`) basado en una clave generada (por ejemplo, mediante un hash MD5) y verificar la expiración de forma consistente en ambos, para evitar reprocesamientos dentro del período definido y maximizar la eficiencia.
- La actualización del historial de URLs procesadas debe implementarse en un módulo independiente (por ejemplo, `history_tracker.py`), ya que esto favorece la reutilización, el testing y la escalabilidad futura.
- La modularización debe preservar la funcionalidad existente del script monolítico (main4_g2.py), separándola en módulos independientes según la estructura propuesta, de modo que el sistema actual siga funcionando sin cambios en su salida.
- Cada módulo debe tener interfaces claras y bien documentadas (por ejemplo, funciones o clases con entradas y salidas definidas) que faciliten la extracción y el despliegue independiente, permitiendo que en el futuro se migre a microservicios si se requiere escalar el sistema.
- Diseñar cada módulo con capacidad de procesamiento asíncrono y paralelo cuando sea posible, para maximizar el rendimiento cuando se procesen PDFs con gran cantidad de enlaces.
- La estructura JSON de salida debe optimizarse considerando su futuro uso para generar embeddings y construir un agente RAG especializado, asegurando que los metadatos y el contenido extraído sean adecuados para este propósito.

Por favor, refactoriza el código del script monolítico (main4_g2.py) dividiéndolo en los módulos indicados (config_manager.py, pdf_processor.py, url_manager.py, html_scraper.py, image_processor.py, api_client.py, cache_utils.py, file_manager.py y history_tracker.py) y crea un orquestador principal en main.py que coordine todo el flujo (desde la extracción del PDF, clasificación de URLs, scraping HTML, extracción de texto de imágenes hasta la consolidación de resultados). Además, genera un archivo .gitignore que excluya credenciales, caché, logs y entornos virtuales."